In [1]:
library(twang)
library(weights)
library(Matching)
library(tidyverse)

Loading required package: gbm
Loading required package: survival
Loading required package: lattice
Loading required package: splines
Loading required package: parallel
Loaded gbm 2.1.3
Loading required package: survey
Loading required package: grid
Loading required package: Matrix

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

Loading required package: xtable
Loading required package: latticeExtra
Loading required package: RColorBrewer
Loading required package: Hmisc
Loading required package: Formula
Loading required package: ggplot2

Attaching package: ‘ggplot2’

The following object is masked from ‘package:latticeExtra’:

    layer


Attaching package: ‘Hmisc’

The following objects are masked from ‘package:xtable’:

    label, label<-

The following object is masked from ‘package:survey’:

    deff

The following objects are masked from ‘package:base’:

    format.pval, units

Loading required package: gdata
gdata: read.xls suppo

In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps_subgroup_vaso.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,1,1,1,1,0,0.7339584,1.362475
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,0,1,1,0,0,1,1,0,0.3889232,2.571202
214106,123152,74869,MICU,2150-09-05 17:50:32,2150-09-12 21:40:59,1,74.13371,1,2150-09-07,⋯,1,1,1,1,1,1,1,1,0.7721262,1.295125
254478,128652,31,MICU,2108-08-22 23:28:42,2108-08-30 21:59:20,1,72.26709,1,NA,⋯,1,1,1,0,1,1,0,1,0.1951075,1.242402
264446,165520,10013,MICU,2125-10-04 23:38:00,2125-10-07 15:13:52,1,87.08742,1,NA,⋯,1,1,1,0,1,0,0,1,0.4902299,1.961669
218143,122936,10370,SICU,2145-01-15 03:36:30,2145-01-20 19:00:30,1,45.06368,1,2145-01-14,⋯,1,1,1,0,1,1,1,0,0.8799575,1.136419


In [4]:
library(tableone)

In [5]:
features <- c("age", "gender", "first_careunit", "weight",
              "saps", "sofa", "elix_score", "vent", "sedative",
              "icd_chf", "icd_afib", "icd_renal", "icd_liver",
              "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
              "icu_adm_weekday", "icu_adm_hour",
              "vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
              "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
              "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
              "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
              "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
              "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag")

In [6]:
tab <- CreateTableOne(vars = features,
                      strata = "echo",
                      data = full_data,
                      argsNormal = list(var.equal = FALSE))
capture.output(tab_df <- tab %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [7]:
tab_df

,0,1,p,SMD
n,839,1228,,
age (mean (sd)),68.63 (16.37),66.26 (16.44),0.001,0.144
gender = F (%),430 (51.3),578 (47.1),0.068,0.084
first_careunit = MICU (%),666 (79.4),982 (80.0),0.787,0.015
weight (mean (sd)),79.51 (23.76),83.75 (28.27),<0.001,0.162
saps (mean (sd)),22.96 (6.23),23.03 (5.27),0.792,0.012
sofa (mean (sd)),8.32 (4.10),8.87 (3.69),0.002,0.141
elix_score (mean (sd)),9.67 (7.48),10.40 (7.82),0.034,0.095
vent = 1 (%),529 (63.1),904 (73.6),<0.001,0.229
sedative = 1 (%),445 (53.0),812 (66.1),<0.001,0.269


In [8]:
ipw_svydesign <- readRDS(file.path(data_dir, "ipw_svydesign_subgroup_vaso.rds"))

In [9]:
tab_wtd <- svyCreateTableOne(vars = features,
                             strata = "echo",
                             data = ipw_svydesign)

In [10]:
capture.output(tab_wtd_df <- tab_wtd %>%
               print %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [11]:
tab_wtd_df

,0,1,p
n,1700.46,1861.66,
age (mean (sd)),67.26 (16.09),66.22 (16.45),0.026
gender = F (%),845.6 (49.7),882.6 (47.4),0.328
first_careunit = MICU (%),1347.4 (79.2),1482.7 (79.6),0.834
weight (mean (sd)),82.01 (24.42),82.89 (25.25),0.122
saps (mean (sd)),23.93 (5.79),24.02 (5.08),0.755
sofa (mean (sd)),9.45 (4.06),9.52 (3.53),0.145
elix_score (mean (sd)),10.21 (7.40),10.46 (7.75),0.635
vent = 1 (%),1115.8 (65.6),1320.1 (70.9),0.017
sedative = 1 (%),963.6 (56.7),1177.3 (63.2),0.005


In [12]:
ps_matches_df <- data.table::fread(file.path(data_dir, "ps_matches_df_subgroup_vaso.csv"), data.table = FALSE)

head(ps_matches_df)

icustay_id,group,match
228283,ctrl,1
249855,ctrl,2
280310,ctrl,3
237390,ctrl,4
293409,ctrl,5
245467,ctrl,6


In [13]:
ps_df <- ps_matches_df %>%
left_join(full_data, by = "icustay_id") %>%
arrange(desc(group), match)

head(ps_df)

icustay_id,group,match,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
214954,trtd,1,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,⋯,1,1,1,1,1,1,1,0,0.7339584,1.362475
264413,trtd,2,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,⋯,0,1,1,0,0,1,1,0,0.3889232,2.571202
214106,trtd,3,123152,74869,MICU,2150-09-05 17:50:32,2150-09-12 21:40:59,1,74.13371,⋯,1,1,1,1,1,1,1,1,0.7721262,1.295125
205397,trtd,4,186819,16488,MICU,2125-11-04 21:46:33,2125-11-05 19:25:20,1,73.27993,⋯,0,1,1,0,0,0,1,0,0.4149716,2.409803
254476,trtd,5,109613,79511,MICU,2176-11-21 02:14:12,2176-11-25 15:50:56,1,75.55016,⋯,0,1,1,0,0,0,1,1,0.5144715,1.943742
208482,trtd,6,138539,18516,MICU,2107-02-16 09:51:48,2107-02-18 17:27:20,1,59.89292,⋯,1,1,1,0,1,1,1,1,0.8413613,1.188550


In [14]:
# df <- ps_df %>%
# select(icu_adm_hour, match, echo) %>%
# filter(!is.na(icu_adm_hour))
# x <- (df %>%
#  filter(echo == 0) %>%
#  rename(ctrl = icu_adm_hour) %>%
#  select(ctrl, match)) %>%
# inner_join(df %>%
#           filter(echo == 1) %>%
#           rename(trtd = icu_adm_hour) %>%
#           select(trtd, match), by = "match")
# tab <- table(x$ctrl, x$trtd, dnn = c("Echo", "Non-Echo"))

In [15]:
# coin::pvalue(tst)
# chisq.test(tab)

In [16]:
tab_ps_pval_df <- features %>% map(function(feature) {
    df <- ps_df %>%
    select(!! as.name(feature), match, echo) %>%
    filter(!is.na(!! as.name(feature)))
    matched <- (df %>% filter(echo == 0) %>% rename(ctrl = (!! as.name(feature))) %>% select(ctrl, match)) %>%
    inner_join(df %>% filter(echo == 1) %>% rename(trtd = (!! as.name(feature))) %>% select(trtd, match),
              by = "match") %>%
    filter(!is.na(ctrl), !is.na(trtd))
    
    if (is.factor(ps_df %>% pull(feature))) {
        tab <- table(matched$trtd, matched$ctrl, dnn = c("Echo", "Non-Echo"))
        if (nlevels(ps_df %>% pull(feature)) > 2) {
            tst <- coin::mh_test(tab)
            p.val <- coin::pvalue(tst)
        } else {
            tst <- mcnemar.test(tab)
            p.val <- tst$p.val
        }
        tst <- mcnemar.test(tab)
    } else {
        tst <- t.test(matched$trtd, matched$ctrl, paired = TRUE)
        p.val <- tst$p.val
    }
    
    data.frame(cov = feature, pval = p.val, stringsAsFactors = FALSE)
}) %>%
data.table::rbindlist() %>%
as.data.frame(stringsAsFactors = FALSE) %>%
mutate(pval = case_when(
    pval < 0.001 ~ "0.001",
    TRUE ~ as.character(round(pval, 3))
)) %>%
rbind(data.frame(cov = "n", pval = "", stringsAsFactors = FALSE), .) %>%
select(cov, pval)

tab_ps_pval_df

cov,pval
n,
age,0.753
gender,0.887
first_careunit,0.864
weight,0.036
saps,0.004
sofa,0.046
elix_score,0.645
vent,0.013
sedative,0.004


In [17]:
tab_ps <- CreateTableOne(vars = features,
                         strata = "echo",
                         data = ps_df)
tab_ps

                                    Stratified by echo
                                     0               1               p     
  n                                     412             412                
  age (mean (sd))                     67.95 (16.67)   67.59 (15.98)   0.751
  gender = F (%)                        203 (49.3)      206 (50.0)    0.889
  first_careunit = MICU (%)             327 (79.4)      330 (80.1)    0.862
  weight (mean (sd))                  80.98 (23.00)   78.05 (21.69)   0.069
  saps (mean (sd))                    23.16 (5.52)    22.09 (5.46)    0.005
  sofa (mean (sd))                     8.68 (3.94)     8.15 (3.66)    0.048
  elix_score (mean (sd))               9.46 (7.19)     9.22 (7.73)    0.651
  vent = 1 (%)                          290 (70.4)      256 (62.1)    0.015
  sedative = 1 (%)                      263 (63.8)      223 (54.1)    0.006
  icd_chf = 1 (%)                       112 (27.2)       67 (16.3)   <0.001
  icd_afib = 1 (%)               

In [18]:
capture.output(tab_ps_df <- tab_ps %>%
               print(smd = TRUE) %>%
               as.data.frame(stringsAsFactors = FALSE) %>%
               select(-test)) %>% invisible

In [19]:
tab_ps_df

,0,1,p,SMD
n,412,412,,
age (mean (sd)),67.95 (16.67),67.59 (15.98),0.751,0.022
gender = F (%),203 (49.3),206 (50.0),0.889,0.015
first_careunit = MICU (%),327 (79.4),330 (80.1),0.862,0.018
weight (mean (sd)),80.98 (23.00),78.05 (21.69),0.069,0.131
saps (mean (sd)),23.16 (5.52),22.09 (5.46),0.005,0.195
sofa (mean (sd)),8.68 (3.94),8.15 (3.66),0.048,0.138
elix_score (mean (sd)),9.46 (7.19),9.22 (7.73),0.651,0.032
vent = 1 (%),290 (70.4),256 (62.1),0.015,0.175
sedative = 1 (%),263 (63.8),223 (54.1),0.006,0.198


In [20]:
wtd_pval <- tab_wtd_df %>%
setNames(c("trtd", "ctrl", "pval")) %>%
mutate(cov = rownames(.)) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov = strsplit(cov, " ") %>% map_chr(first)) %>%
filter(!is.null(pval) & !is.na(pval) & pval != "") %>%
mutate(pval = str_replace_all(pval, "<", "")) %>%
mutate(pval = as.numeric(pval)) %>%
select(cov, pval)

wtd_pval

cov,pval
age,0.026
gender,0.328
first_careunit,0.834
weight,0.122
saps,0.755
sofa,0.145
elix_score,0.635
vent,0.017
sedative,0.005
icd_chf,0.001


In [21]:
tab_fmt <- tab_df %>%
setNames(c("ctrl", "trtd", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, pval, smd)

tab_fmt

cov,ctrl,trtd,pval,smd
n,839,1228,,
age (mean (sd)),68.63 (16.37),66.26 (16.44),0.001,0.144
gender = F (%),51.3%,47.1%,0.068,0.084
first_careunit = MICU (%),79.4%,80.0%,0.787,0.015
weight (mean (sd)),79.51 (23.76),83.75 (28.27),<0.001,0.162
saps (mean (sd)),22.96 (6.23),23.03 (5.27),0.792,0.012
sofa (mean (sd)),8.32 (4.10),8.87 (3.69),0.002,0.141
elix_score (mean (sd)),9.67 (7.48),10.40 (7.82),0.034,0.095
vent = 1 (%),63.1%,73.6%,<0.001,0.229
sedative = 1 (%),53.0%,66.1%,<0.001,0.269


In [22]:
tab_ps_fmt <- tab_ps_df %>%
setNames(c("ctrl", "trtd", "pval", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, smd)

tab_ps_fmt

cov,ctrl,trtd,smd
n,412,412,
age (mean (sd)),67.95 (16.67),67.59 (15.98),0.022
gender = F (%),49.3%,50.0%,0.015
first_careunit = MICU (%),79.4%,80.1%,0.018
weight (mean (sd)),80.98 (23.00),78.05 (21.69),0.131
saps (mean (sd)),23.16 (5.52),22.09 (5.46),0.195
sofa (mean (sd)),8.68 (3.94),8.15 (3.66),0.138
elix_score (mean (sd)),9.46 (7.19),9.22 (7.73),0.032
vent = 1 (%),70.4%,62.1%,0.175
sedative = 1 (%),63.8%,54.1%,0.198


In [23]:
tab_wtd_fmt <- tab_wtd_df %>%
setNames(c("ctrl", "trtd", "pval")) %>%
mutate(cov = rownames(.)) %>%
mutate(ctrl = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(ctrl, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     ctrl)) %>%
mutate(trtd = ifelse((grepl("%", cov) & grepl("=", cov)) | grepl("^\\s+", cov),
                     str_extract(trtd, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     trtd)) %>%
select(cov, ctrl, trtd, pval)

tab_wtd_fmt

cov,ctrl,trtd,pval
n,1700.46,1861.66,
age (mean (sd)),67.26 (16.09),66.22 (16.45),0.026
gender = F (%),49.7%,47.4%,0.328
first_careunit = MICU (%),79.2%,79.6%,0.834
weight (mean (sd)),82.01 (24.42),82.89 (25.25),0.122
saps (mean (sd)),23.93 (5.79),24.02 (5.08),0.755
sofa (mean (sd)),9.45 (4.06),9.52 (3.53),0.145
elix_score (mean (sd)),10.21 (7.40),10.46 (7.75),0.635
vent = 1 (%),65.6%,70.9%,0.017
sedative = 1 (%),56.7%,63.2%,0.005


In [24]:
tab_raw <-
tab_fmt %>%
full_join(tab_ps_fmt, by = "cov") %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov = strsplit(cov, " ") %>% map_chr(first)) %>%
left_join(tab_ps_pval_df, by = "cov")

tab_raw

cov,ctrl.x,trtd.x,pval.x,smd.x,ctrl.y,trtd.y,smd.y,pval.y
n,839,1228,,,412,412,,
age,68.63 (16.37),66.26 (16.44),0.001,0.144,67.95 (16.67),67.59 (15.98),0.022,0.753
gender,51.3%,47.1%,0.068,0.084,49.3%,50.0%,0.015,0.887
first_careunit,79.4%,80.0%,0.787,0.015,79.4%,80.1%,0.018,0.864
weight,79.51 (23.76),83.75 (28.27),<0.001,0.162,80.98 (23.00),78.05 (21.69),0.131,0.036
saps,22.96 (6.23),23.03 (5.27),0.792,0.012,23.16 (5.52),22.09 (5.46),0.195,0.004
sofa,8.32 (4.10),8.87 (3.69),0.002,0.141,8.68 (3.94),8.15 (3.66),0.138,0.046
elix_score,9.67 (7.48),10.40 (7.82),0.034,0.095,9.46 (7.19),9.22 (7.73),0.032,0.645
vent,63.1%,73.6%,<0.001,0.229,70.4%,62.1%,0.175,0.013
sedative,53.0%,66.1%,<0.001,0.269,63.8%,54.1%,0.198,0.004


In [25]:
tab_full_fmt <-
tab_fmt %>%
setNames(c("cov", "Non-Echo", "Echo", "p value", "SMD")) %>%
full_join(tab_wtd_fmt %>%
          setNames(c("cov", "Non-Echo (weighted cohort)", "Echo (weighted cohort)",
                     "p value (weighted cohort)")), by = "cov") %>%
full_join(tab_ps_fmt %>%
          setNames(c("cov", "Non-Echo (PS matching)", "Echo (PS matching)",
                     "SMD (PS matching)")), by = "cov") %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_ = strsplit(cov, " ") %>% map_chr(first)) %>%
left_join(tab_ps_pval_df %>%
          setNames(c("cov_", "p value (PS matching)")), by = "cov_") %>%
mutate(cov = str_replace_all(cov_, "_", " ")) %>%
mutate(cov = str_replace_all(cov, "lab|vs|flag|icd|first", "")) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov = case_when(
    grepl("icd|vs|lab|day", cov_) ~ toupper(cov),
    TRUE ~ tools::toTitleCase(cov)
)) %>%
mutate(cov = case_when(
    cov == "Gender" ~ "Gender (Female)",
    cov == "Careunit" ~ "Service Unit (MICU vs SICU)",
    cov == "Weight" ~ "Weight (Kg)",
    cov == "Elix Score" ~ "Elixhauser Score",
    cov == "Saps" ~ "SAPS Score",
    cov == "Sofa" ~ "SOFA Score",
    cov == "Vent" ~ "Mechanical Ventilation Use (1st 24 Hours)",
    cov == "Vaso" ~ "Vasopressor Use (1st 24 Hours)",
    cov == "Sedative" ~ "Sedative Use (1st 24 Hours)",
    cov == "ICU ADM WEEKDAY" ~ "Day of ICU Admission",
    cov == "Icu Adm Hour" ~ "Hour of ICU Admission",
    cov == "TEMP" ~ "Temperature (C)",
    cov == "HEART RATE" ~ "Heart Rate",
    cov %in% c("TROPONIN", "BNP", "CREATININE KINASE") ~ paste(cov, "(tested)"),
    TRUE ~ cov
)) %>%
select(-cov_) %>%
rename(Covariate = cov) %>%
select(1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 11, 12)

tab_full_fmt

Covariate,Non-Echo,Echo,SMD,p value,Non-Echo (weighted cohort),Echo (weighted cohort),p value (weighted cohort),Non-Echo (PS matching),Echo (PS matching),SMD (PS matching),p value (PS matching)
n,839,1228,,,1700.46,1861.66,,412,412,,
Age,68.63 (16.37),66.26 (16.44),0.144,0.001,67.26 (16.09),66.22 (16.45),0.026,67.95 (16.67),67.59 (15.98),0.022,0.753
Gender (Female),51.3%,47.1%,0.084,0.068,49.7%,47.4%,0.328,49.3%,50.0%,0.015,0.887
Service Unit (MICU vs SICU),79.4%,80.0%,0.015,0.787,79.2%,79.6%,0.834,79.4%,80.1%,0.018,0.864
Weight (Kg),79.51 (23.76),83.75 (28.27),0.162,<0.001,82.01 (24.42),82.89 (25.25),0.122,80.98 (23.00),78.05 (21.69),0.131,0.036
SAPS Score,22.96 (6.23),23.03 (5.27),0.012,0.792,23.93 (5.79),24.02 (5.08),0.755,23.16 (5.52),22.09 (5.46),0.195,0.004
SOFA Score,8.32 (4.10),8.87 (3.69),0.141,0.002,9.45 (4.06),9.52 (3.53),0.145,8.68 (3.94),8.15 (3.66),0.138,0.046
Elixhauser Score,9.67 (7.48),10.40 (7.82),0.095,0.034,10.21 (7.40),10.46 (7.75),0.635,9.46 (7.19),9.22 (7.73),0.032,0.645
Mechanical Ventilation Use (1st 24 Hours),63.1%,73.6%,0.229,<0.001,65.6%,70.9%,0.017,70.4%,62.1%,0.175,0.013
Sedative Use (1st 24 Hours),53.0%,66.1%,0.269,<0.001,56.7%,63.2%,0.005,63.8%,54.1%,0.198,0.004


In [26]:
tab_report <- tab_full_fmt %>%
filter(! Covariate %in% as.character(0:24)) %>%
select(1, 2, 3, 4, 9, 10, 11)

tab_report

Covariate,Non-Echo,Echo,SMD,Non-Echo (PS matching),Echo (PS matching),SMD (PS matching)
n,839,1228,,412,412,
Age,68.63 (16.37),66.26 (16.44),0.144,67.95 (16.67),67.59 (15.98),0.022
Gender (Female),51.3%,47.1%,0.084,49.3%,50.0%,0.015
Service Unit (MICU vs SICU),79.4%,80.0%,0.015,79.4%,80.1%,0.018
Weight (Kg),79.51 (23.76),83.75 (28.27),0.162,80.98 (23.00),78.05 (21.69),0.131
SAPS Score,22.96 (6.23),23.03 (5.27),0.012,23.16 (5.52),22.09 (5.46),0.195
SOFA Score,8.32 (4.10),8.87 (3.69),0.141,8.68 (3.94),8.15 (3.66),0.138
Elixhauser Score,9.67 (7.48),10.40 (7.82),0.095,9.46 (7.19),9.22 (7.73),0.032
Mechanical Ventilation Use (1st 24 Hours),63.1%,73.6%,0.229,70.4%,62.1%,0.175
Sedative Use (1st 24 Hours),53.0%,66.1%,0.269,63.8%,54.1%,0.198


In [27]:
data.table::fwrite(tab_full_fmt, file.path(data_dir, "tableone_full_subgroup_vaso.csv"))
data.table::fwrite(tab_raw, file.path(data_dir, "tableone_raw_subgroup_vaso.csv"))
data.table::fwrite(tab_report, file.path(data_dir, "tableone_report_subgroup_vaso.csv"))
data.table::fwrite(wtd_pval, file.path(data_dir, "wtd_pval_subgroup_vaso.csv"))